# audio

> Route handlers for audio playback controls

In [ ]:
#| default_exp routes.audio

In [ ]:
#| export
from typing import Dict, Callable, Tuple

from fasthtml.common import APIRouter, Script

from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_transcript_review.models import ReviewUrls
from cjm_transcript_review.routes.core import (
    WorkflowStateStore, _update_review_state
)

# Debug flag
DEBUG_AUDIO_ROUTES = False

## Speed Change Handler

Updates playback speed state and returns JavaScript to change the Web Audio API playback rate.

In [ ]:
#| export
def _generate_speed_change_js(
    speed:float  # New playback speed
) -> str:  # JavaScript to update playback rate
    """Generate JS to update Web Audio API playback rate."""
    return f"""
        window._reviewPlaybackSpeed = {speed};
        if (window.DEBUG_REVIEW_AUDIO) {{
            console.log('[REVIEW_AUDIO] Playback speed changed to:', {speed});
        }}
    """

## Auto-Navigate Toggle Handler

Updates auto-navigate state and returns JavaScript to update the client-side flag.

In [ ]:
#| export
def _generate_auto_nav_js(
    enabled:bool  # Whether auto-navigate is enabled
) -> str:  # JavaScript to update auto-navigate flag
    """Generate JS to update auto-navigate client-side flag."""
    enabled_str = "true" if enabled else "false"
    return f"""
        window._reviewAutoNavigate = {enabled_str};
        if (window.DEBUG_REVIEW_AUDIO) {{
            console.log('[REVIEW_AUDIO] Auto-navigate changed to:', {enabled_str});
        }}
    """

## Replay Current Handler

Returns JavaScript to replay the current segment's audio. No state change needed.

In [ ]:
#| export
def _generate_replay_js() -> str:  # JavaScript to replay current segment
    """Generate JS to replay the current segment's audio."""
    return """
        if (window.replayReviewSegment) {
            window.replayReviewSegment();
        } else if (window.DEBUG_REVIEW_AUDIO) {
            console.log('[REVIEW_AUDIO] replayReviewSegment not defined yet');
        }
    """

## Router Initialization

Creates audio control routes and returns the router with route functions.

In [ ]:
#| export
def init_audio_router(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    prefix:str,  # Base prefix for audio routes
    urls:ReviewUrls,  # URL bundle to populate
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, routes dict)
    """Initialize audio control routes."""
    router = APIRouter(prefix=prefix)
    routes = {}
    
    @router.post("/speed_change")
    def speed_change(request, sess, speed:float):
        """Handle playback speed change."""
        session_id = get_session_id(sess)
        if DEBUG_AUDIO_ROUTES:
            print(f"[AUDIO_ROUTES] speed_change: {speed}")
        
        # Update state
        _update_review_state(
            state_store, workflow_id, session_id,
            playback_speed=speed
        )
        
        # Return JS to update client-side playback rate
        return Script(_generate_speed_change_js(speed))
    
    routes["speed_change"] = speed_change
    
    @router.post("/toggle_auto_nav")
    async def toggle_auto_nav(request, sess):
        """Handle auto-navigate toggle."""
        session_id = get_session_id(sess)
        
        # Checkbox only sends value when checked
        form_data = await request.form()
        auto_navigate = "auto_navigate" in form_data
        
        if DEBUG_AUDIO_ROUTES:
            print(f"[AUDIO_ROUTES] toggle_auto_nav: {auto_navigate}")
        
        # Update state
        _update_review_state(
            state_store, workflow_id, session_id,
            auto_navigate=auto_navigate
        )
        
        # Return JS to update client-side flag
        return Script(_generate_auto_nav_js(auto_navigate))
    
    routes["toggle_auto_nav"] = toggle_auto_nav
    
    @router.post("/replay_current")
    def replay_current(request, sess):
        """Handle replay current segment request."""
        if DEBUG_AUDIO_ROUTES:
            print(f"[AUDIO_ROUTES] replay_current")
        
        # Return JS to replay current segment
        return Script(_generate_replay_js())
    
    routes["replay_current"] = replay_current
    
    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()